In [1]:
from llama_index.core import (
    Settings,
    VectorStoreIndex,
    SimpleDirectoryReader,
)

import torch
from transformers import AutoTokenizer, AutoModel
from llama_index.core import Settings, VectorStoreIndex
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import shutil
from llama_parse import LlamaParse
from llama_index.core.node_parser import SentenceSplitter
import os
from typing import List, Optional
from typing import List, Optional
from pydantic import BaseModel, EmailStr, Field, root_validator, field_validator
from dotenv import load_dotenv

load_dotenv()
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
import openai
import numpy as np
import json
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    Settings,
    Document,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.node_parser import MarkdownElementNodeParser
from sklearn.feature_extraction.text import TfidfVectorizer
import streamlit as st
from transformers import AutoTokenizer, AutoModel
import torch

c:\Users\Admin\anaconda3\envs\llamaindex\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import nest_asyncio; nest_asyncio.apply()

In [3]:
LLAMA_CLOUD_API_KEY = st.secrets["LLAMA_CLOUD_API_KEY"]

In [4]:
# Pydantic model for extracting education
class Education(BaseModel):
    institution: Optional[str] = Field(
        None, description="The name of the educational institution"
    )
    degree: Optional[str] = Field(
        None, description="The degree or qualification earned"
    )
    graduation_date: Optional[str] = Field(
        None, description="The graduation date (e.g., 'YYYY-MM')"
    )
    details: Optional[List[str]] = Field(
        None,
        description="Additional details about the education (e.g., coursework, achievements)",
    )

    @field_validator("details", mode="before")
    def validate_details(cls, v):
        if isinstance(v, str) and v.lower() == "n/a":
            return []
        elif not isinstance(v, list):
            return []
        return v


# Pydantic model for extracting experience
class Experience(BaseModel):
    company: Optional[str] = Field(
        None, description="The name of the company or organization"
    )
    location: Optional[str] = Field(
        None, description="The location of the company or organization"
    )
    role: Optional[str] = Field(
        None, description="The role or job title held by the candidate"
    )
    start_date: Optional[str] = Field(
        None, description="The start date of the job (e.g., 'YYYY-MM')"
    )
    end_date: Optional[str] = Field(
        None,
        description="The end date of the job or 'Present' if ongoing (e.g., 'MM-YYYY')",
    )
    responsibilities: Optional[List[str]] = Field(
        None, description="A list of responsibilities and tasks handled during the job"
    )

    @field_validator("responsibilities", mode="before")
    def validate_responsibilities(cls, v):
        if isinstance(v, str) and v.lower() == "n/a":
            return []
        elif not isinstance(v, list):
            return []
        return v


# Main Pydantic class ensapsulating education and epxerience classes with other information
class Candidate(BaseModel):
    name: Optional[str] = Field(None, description="The full name of the candidate")
    email: Optional[EmailStr] = Field(None, description="The email of the candidate")
    age: Optional[int] = Field(None, description="The age of the candidate.")
    skills: Optional[List[str]] = Field(
        None, description="A list of high-level skills possessed by the candidate."
    )
    experience: Optional[List[Experience]] = Field(
        None,
        description="A list of experiences detailing previous jobs, roles, and responsibilities",
    )
    education: Optional[List[Education]] = Field(
        None,
        description="A list of educational qualifications of the candidate including degrees, institutions studied in, and dates of start and end.",
    )

    @root_validator(pre=True)
    def handle_invalid_values(cls, values):
        for key, value in values.items():
            if isinstance(value, str) and value.lower() in {"n/a", "none", ""}:
                values[key] = None
        return values

C:\Users\Admin\AppData\Local\Temp\ipykernel_16344\2887394112.py:74: PydanticDeprecatedSince20: Pydantic V1 style `@root_validator` validators are deprecated. You should migrate to Pydantic V2 style `@model_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  @root_validator(pre=True)


In [21]:
import os
from dotenv import load_dotenv
import re

# Load environment variables and set OpenAI API key
load_dotenv()

True

In [27]:
from llama_index.llms.ollama import Ollama

# llm = Ollama(model="llama3.2:1b", request_timeout=300.0)
llm = Ollama(model="llama3.2", request_timeout=300.0)

In [28]:
response = llm.complete("What is the capital of France?")
print(response)

The capital of France is Paris.


In [29]:
llm_option = llm
file_path = "./Dang_Quang_Dung_CV__Copy___Copy_.pdf"

"""
Extracts candidate data from the resume.
"""
print(f"Extracting CV data. LLM: {llm_option}")
output_schema = Candidate.model_json_schema()
parser = LlamaParse(
    result_type="markdown",
    parsing_instructions="Extract each section separately based on the document structure.",
    premium_mode=True,
    # api_key=os.getenv("LLAMA_CLOUD_API_KEY"),
    api_key="llx-OxbevJhMJFNaEWwoc5MstJ88kOn3s1ane6hwGzRzbSogArUS",
    verbose=True,
)
file_extractor = {".pdf": parser}

# Load resume
documents = SimpleDirectoryReader(
    input_files=[file_path], file_extractor=file_extractor
).load_data()



Extracting CV data. LLM: callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x000002450DECF310> system_prompt=None messages_to_prompt=<function messages_to_prompt at 0x0000024500E4F0A0> completion_to_prompt=<function default_completion_to_prompt at 0x0000024500EDAB00> output_parser=None pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'> query_wrapper_prompt=None base_url='http://localhost:11434' model='llama3.2' temperature=0.75 context_window=3900 request_timeout=300.0 prompt_key='prompt' json_mode=False additional_kwargs={}
Error while parsing the PDF file:  Failed to parse the PDF file: {"detail":[{"type":"enum","loc":["body","language",0],"msg":"Input should be 'af', 'az', 'bs', 'cs', 'cy', 'da', 'de', 'en', 'es', 'et', 'fr', 'ga', 'hr', 'hu', 'id', 'is', 'it', 'ku', 'la', 'lt', 'lv', 'mi', 'ms', 'mt', 'nl', 'no', 'oc', 'pi', 'pl', 'pt', 'ro', 'rs_latin', 'sk', 'sl', 'sq', 'sv', 'sw', 'tl', 'tr', 'uz', 'vi', 'ar', 'fa', 'ug', 'ur', 'bn', 'as', '

In [30]:
llm_option = llm
file_path = "./Dang_Quang_Dung_CV__Copy___Copy_.pdf"

"""
Extracts candidate data from the resume.
"""
print(f"Extracting CV data. LLM: {llm_option}")
output_schema = Candidate.model_json_schema()
# parser = LlamaParse(
#     result_type="markdown",
#     parsing_instructions="Extract each section separately based on the document structure.",
#     premium_mode=True,
#     # api_key=os.getenv("LLAMA_CLOUD_API_KEY"),
#     api_key="llx-OxbevJhMJFNaEWwoc5MstJ88kOn3s1ane6hwGzRzbSogArUS",
#     verbose=True,
# )
# file_extractor = {".pdf": parser}

# Load resume
documents = SimpleDirectoryReader(
    input_files=[file_path]
).load_data()



Extracting CV data. LLM: callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x000002450DECF310> system_prompt=None messages_to_prompt=<function messages_to_prompt at 0x0000024500E4F0A0> completion_to_prompt=<function default_completion_to_prompt at 0x0000024500EDAB00> output_parser=None pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'> query_wrapper_prompt=None base_url='http://localhost:11434' model='llama3.2' temperature=0.75 context_window=3900 request_timeout=300.0 prompt_key='prompt' json_mode=False additional_kwargs={}


In [31]:
documents

[Document(id_='0fb220c4-cdbf-40ba-999a-95556bf8da70', embedding=None, metadata={'page_label': '1', 'file_name': 'e:/Master-code/Job-recommend/CV-Analyzer-Job-Recommender/Multiple models/Dang_Quang_Dung_CV__Copy___Copy_.pdf', 'file_path': 'Dang_Quang_Dung_CV__Copy___Copy_.pdf', 'file_type': 'application/pdf', 'file_size': 763671, 'creation_date': '2025-01-19', 'last_modified_date': '2024-12-30', 'last_accessed_date': None}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Dang Quang Dung\nanhdungvk01@gmail.com\n0869040236\nLinkedin\nGithub\nCau Giay - Ha Noi\nProfile\nInﬁveyears’time,IwanttobecomeanAIEngineer.MystartingpointistobeginthispositionasaSoftwareEngineer.\nEducation\n09/2019–05/2024 PostsandTelecommunicationsInstituteofTechnology .\nInformationTe

In [32]:
_resume_content = "\n".join([doc.text for doc in documents])

In [33]:
_resume_content

'Dang Quang Dung\nanhdungvk01@gmail.com\n0869040236\nLinkedin\nGithub\nCau Giay - Ha Noi\nProfile\nInﬁveyears’time,IwanttobecomeanAIEngineer.MystartingpointistobeginthispositionasaSoftwareEngineer.\nEducation\n09/2019–05/2024 PostsandTelecommunicationsInstituteofTechnology .\nInformationTechnology\nGPA3.31/4.0\nResearch Interests\nNaturalLanguageProcessing(NLP),MachineLearningandDeepLearning.\nWord Experience\nMasterTraineratCentreforDevelopmentofAdvancedComputing(C-DAC),India(09/2024-\npresent)\n•ParticipateinArtiﬁcialIntelligence&DataScience:PG-DBDA-PostGraduateDiplomainBigData\nAnalytics\nAIResearcheratReseachInstituteofPostsandTelecommunications(RIPT)-PTIT(08/2024-\npresent)\n•Researchandwritingpaper\n•BuildachatbotsystemusingRAGtechniques,FunctionCallingwithLLMs\nSoftwareAIEngineeratA.I-SoftJSC(02/2023-08/2024)\n•Buildwebsitesforresearchtopics.\n•Buildachatbotsystemtosupportenrollment(End-to-End)usingRasaandBotpressFramework.\n•Buildacrawldatamodule,dataanalysisusingBERTopicforsoc

In [26]:
output_schema

{'$defs': {'Education': {'properties': {'institution': {'anyOf': [{'type': 'string'},
      {'type': 'null'}],
     'default': None,
     'description': 'The name of the educational institution',
     'title': 'Institution'},
    'degree': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'description': 'The degree or qualification earned',
     'title': 'Degree'},
    'graduation_date': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'description': "The graduation date (e.g., 'YYYY-MM')",
     'title': 'Graduation Date'},
    'details': {'anyOf': [{'items': {'type': 'string'}, 'type': 'array'},
      {'type': 'null'}],
     'default': None,
     'description': 'Additional details about the education (e.g., coursework, achievements)',
     'title': 'Details'}},
   'title': 'Education',
   'type': 'object'},
  'Experience': {'properties': {'company': {'anyOf': [{'type': 'string'},
      {'type': 'null'}],
     'default': None,
    

In [34]:
# Store the pre-extracted content

prompt = f"""
    You are an expert in analyzing resumes. Use the following JSON schema to extract relevant information:
    ```json
    {output_schema}
    ```json
    Extract the information from the following document and provide a structured JSON response strictly adhering to the schema above. 
    Please remove any ```json ``` characters from the output. Do not make up any information. If a field cannot be extracted, mark it as `n/a`.
    Document:
    ----------------
    {_resume_content}
    ----------------
    """
try:
    response = llm_option.complete(prompt)
    if not response or not response.text:
        raise ValueError("Failed to get a response from LLM.")

    parsed_data = json.loads(response.text)
    print(Candidate.model_validate(parsed_data))
except Exception as e:
    print(f"Error parsing response: {str(e)}")
    raise ValueError(
        "Failed to extract insights. Please ensure the resume and query engine are properly configured."
    )

Error parsing response: Expecting ',' delimiter: line 7 column 3 (char 1412)


ValueError: Failed to extract insights. Please ensure the resume and query engine are properly configured.

In [39]:
response.text

'{\n  "name": "Dang Quang Dung",\n  "email": "anhdungvk01@gmail.com",\n  "age": "n/a",\n  "skills": "[\\"English\\", \\"Vietnamese (native speaker)\\", \\"Python\\", \\"Linux\\", \\"SQL\\", \\"FastAPI\\", \\"Docker\\", ..., \\"BERTopic\\", \\"Rasa\\", \\"Botpress\\", \\"Langchain\\", \\"LlamaIndex\\", \\"TensorFlow\\", \\"Keras\\", \\"PyTorch\\"]",\n  "experience": "[{\\"company\\": \\"Cau Giay - Ha Noi\\", \\"location\\": \\"Profile\\", \\"role\\": \\"Software Engineer\\", \\"start_date\\": \\"n/a\\", \\"end_date\\": \\"Present\\", \\"responsibilities\\": \\"n/a\\"}, {\\"company\\": \\"CentreforDevelopmentofAdvancedComputing (C-DAC), India\\", \\"location\\": \\"MasterTrainer role at C-DAC, India\\", \\"role\\": \\"AI Researcher and Master Trainer\\", \\"start_date\\": \\"09/2024\\", \\"end_date\\": \\"Present\\", \\"responsibilities\\": \\"Research and writing paper\\"}, {\\"company\\": \\"A.I-SoftJSC\\", \\"location\\": \\"Software AI Engineer at A.I-SoftJSC\\", \\"role\\": \\"Softw

In [40]:
parsed_data = json.loads(response.text)
parsed_data

JSONDecodeError: Expecting ',' delimiter: line 7 column 3 (char 1412)

: 

In [38]:
print(Candidate.model_validate(parsed_data))

NameError: name 'parsed_data' is not defined